In [2]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.use('Qt5Agg')
sns.set_style("whitegrid")

import calendar

# DATA PREPROCESSING

### MSTA

In [3]:
MSTA= pd.read_excel('Data_36544086.xlsx', sheet_name='MSTA',
                                header=0, 
                                usecols=['Time', 'Anomaly (deg C)'],
                                parse_dates=True).squeeze()

In [4]:
MSTA['Time'] = pd.to_datetime(MSTA['Time'])
MSTA = MSTA.set_index('Time')

In [5]:
MSTA.describe()

,Anomaly (deg C)
count,2100.000000
mean,-0.065022
std,0.406237
min,-1.044895
25%,-0.345928
50%,-0.153942
75%,0.122397
max,1.352173


In [6]:
MSTA.isna().sum()

Anomaly (deg C)    0
dtype: int64

In [7]:
# Extract data from the year 1950 onwards
MSTA_1950_onwards = MSTA[MSTA.index >= '1950-01-01']
MSTA_1950_onwards

,Anomaly (deg C)
Time,
1950-01-01,-0.300044
1950-02-01,-0.370036
1950-03-01,-0.216438
1950-04-01,-0.243616
1950-05-01,-0.137298
...,...
2024-08-01,1.239584
2024-09-01,1.144937
2024-10-01,1.199982


### CH4

In [8]:
CH4 = pd.read_excel('Data_36544086.xlsx', sheet_name='CH4',
                header=0, 
                usecols=['Year', 'Month', 'NOAA CH4 (ppb)'], 
                parse_dates=True).squeeze()

In [9]:
# Convert 'Year' and 'Month' to strings and concatenate them
CH4['Time'] = CH4['Year'].astype(str) + '-' + CH4['Month'].astype(str).str.zfill(2)

In [10]:
CH4 = CH4.drop(columns=['Year', 'Month'])
CH4['Time'] = pd.to_datetime(CH4['Time'])
CH4 = CH4.set_index('Time')


In [11]:
CH4.describe()
CH4.isna().sum()

NOAA CH4 (ppb)    0
dtype: int64

In [12]:
CH4

,NOAA CH4 (ppb)
Time,
1983-07-01,1625.95
1983-08-01,1628.05
1983-09-01,1638.44
1983-10-01,1644.80
1983-11-01,1642.59
...,...
2024-05-01,1926.36
2024-06-01,1921.77
2024-07-01,1921.03


### GMAF

In [13]:
GMAF = pd.read_excel('Data_36544086.xlsx', sheet_name='GMAF',
                    header=None,
                    skiprows=227,
                    usecols=[0,2],
                    names=['Time', 'Passenger Count'], 
                    parse_dates=True).squeeze()

In [14]:
# Reset the index to access the 'Time' column
#GMAF = GMAF.reset_index()

# Convert the 'Time' column to datetime format
GMAF['Time'] = pd.to_datetime(GMAF['Time'], format='%Y %b')

# Format the dates to 'YYYY-MM'
GMAF['Time'] = GMAF['Time'].dt.strftime('%Y-%m')

GMAF['Time'] = pd.to_datetime(GMAF['Time'])

# Set 'Time' column back as the index
GMAF = GMAF.set_index('Time')

In [15]:
GMAF['Passenger Count'] = GMAF['Passenger Count'].astype(int)

In [16]:
GMAF.describe()

,Passenger Count
count,528.000000
mean,4259.448864
std,2304.916442
min,150.000000
25%,2475.500000
50%,4033.000000
75%,5793.250000
max,11628.000000


In [17]:
GMAF.isna().sum()

Passenger Count    0
dtype: int64

### ET12

In [18]:
ET12 = pd.read_excel('Data_36544086.xlsx', sheet_name='ET12',
                                    header=None,
                                    skiprows= 6,
                                    usecols=[0,1],
                                    names=['Time', 'Unadjusted Total'], parse_dates=True).squeeze()

In [19]:
ET12 = ET12.set_index('Time')

In [20]:
ET12['Unadjusted Total'] = ET12['Unadjusted Total'].round(2)

In [21]:
# Reset the index to access the 'Time' column
ET12 = ET12.reset_index()

# Clean the 'Time' column to remove unwanted characters or spaces
ET12['Time'] = ET12['Time'].str.replace(r'\[.*\]', '', regex=True).str.strip()

# Convert the 'Time' column to datetime format
ET12['Time'] = pd.to_datetime(ET12['Time'], format='%B %Y')

# Format the dates to 'YYYY-MM'
ET12['Time'] = ET12['Time'].dt.strftime('%Y-%m')

# Set 'Time' column back as the index
ET12['Time'] = pd.to_datetime(ET12['Time'])
ET12 = ET12.set_index('Time')

In [22]:
ET12.describe()

,Unadjusted Total
count,360.000000
mean,17.369639
std,3.374263
min,10.020000
25%,14.890000
50%,16.900000
75%,19.735000
max,25.020000


In [23]:
ET12.isna().sum()

Unadjusted Total    0
dtype: int64

### Calendar Adjustment

In [24]:
variables = [MSTA_1950_onwards, CH4, GMAF, ET12]

In [25]:
#Calendar adjustment
# Create empty column for days in the month
def calendar_adjustment(df):
    df["Days"] = np.nan
    # Fill empty columns with the days per calendar month
    for date in df.index:
        df.loc[date, "Days"] = calendar.monthrange(date.year, date.month)[1]
    # Perform calendar adjustment
    df["Adjusted Data"] = (df.iloc[:, 0] * 365.25 / (12 * df["Days"])).round(2)


#for i in variables:
for i in [MSTA_1950_onwards, GMAF, ET12]:
    calendar_adjustment(i)

C:\Users\admin\AppData\Local\Temp\ipykernel_22100\3493902977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Days"] = np.nan
C:\Users\admin\AppData\Local\Temp\ipykernel_22100\3493902977.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Adjusted Data"] = (df.iloc[:, 0] * 365.25 / (12 * df["Days"])).round(2)


In [26]:
# CH4 = CH4.drop(columns=['Days', 'NOAA CH4 (ppb)'])
MSTA_1950_onwards = MSTA_1950_onwards.drop(columns=['Days', 'Anomaly (deg C)'])
GMAF = GMAF.drop(columns=['Days', 'Passenger Count'])
ET12 = ET12.drop(columns=['Days', 'Unadjusted Total'])

In [27]:
def rename_columns(df, df_name):
    df = df.rename(columns={'Adjusted Data': df_name})
    return df

In [28]:
# CH4 = rename_columns(CH4, 'CH4 (adjusted)')
MSTA_1950_onwards = rename_columns(MSTA_1950_onwards, 'Anomaly (adjusted deg C)')#
GMAF = rename_columns(GMAF, 'Passenger Count (adjusted)')
ET12 = rename_columns(ET12, 'Total ET12 (adjusted)')

In [29]:
CH4

,NOAA CH4 (ppb)
Time,
1983-07-01,1625.95
1983-08-01,1628.05
1983-09-01,1638.44
1983-10-01,1644.80
1983-11-01,1642.59
...,...
2024-05-01,1926.36
2024-06-01,1921.77
2024-07-01,1921.03


In [30]:
# Add frequency to the index
def frequency_add(i):
    i.index = pd.DatetimeIndex(i.index.values, freq=i.index.inferred_freq)

for i in variables:
    frequency_add(i)

In [31]:
# Set the name attribute for each DataFrame
variables[0].name = 'MSTA_1950_onwards'
variables[1].name = 'CH4'
variables[2].name = 'GMAF'
variables[3].name = 'ET12'

### Merged Data

In [32]:
MSTA_1950_onwards.columns.tolist()

['Anomaly (adjusted deg C)']

In [33]:
merged_df = pd.merge(MSTA_1950_onwards, CH4, left_index=True, right_index=True, how='inner')
merged_df = pd.merge(merged_df, GMAF, left_index=True, right_index=True, how='inner')
merged_df = pd.merge(merged_df, ET12, left_index=True, right_index=True, how='inner')
merged_df = merged_df.round(2)

merged_df.isna().sum()



Anomaly (adjusted deg C)      0
NOAA CH4 (ppb)                0
Passenger Count (adjusted)    0
Total ET12 (adjusted)         0
dtype: int64

In [34]:
merged_df

,Anomaly (adjusted deg C),NOAA CH4 (ppb),Passenger Count (adjusted),Total ET12 (adjusted)
1995-01-01,0.41,1751.12,2225.86,20.65
1995-02-01,0.70,1749.48,2399.13,21.74
1995-03-01,0.35,1749.01,2662.79,22.13
1995-04-01,0.43,1749.43,3407.99,17.84
1995-05-01,0.26,1747.63,3422.75,16.13
...,...,...,...,...
2023-08-01,1.18,1917.22,10037.50,11.28
2023-09-01,1.37,1925.06,9615.21,11.87
2023-10-01,1.26,1930.72,8339.88,13.02
2023-11-01,1.35,1931.85,5794.29,15.37


In [35]:
# Extract the year and month from the index
merged_df['year'] = merged_df.index.year
merged_df['month'] = merged_df.index.month  

# Count the number of unique months for each year
month_counts = merged_df.groupby('year')['month'].nunique()

# Convert to DataFrame for a better representation
month_counts = pd.DataFrame(month_counts).reset_index()
month_counts.columns = ['year', 'unique_month_count']

month_counts



,year,unique_month_count
0,1995,12
1,1996,12
2,1997,12
3,1998,12
4,1999,12
5,2000,12
6,2001,12
7,2002,12
8,2003,12
9,2004,12


### Adding Auxiliary Variables

In [36]:
#Adding month variables D1-D12 
for month in range(1, 13):
    merged_df[f'D{month}'] = 0


merged_df['month'] = merged_df.index.month  


for month in range(1, 13):
    merged_df.loc[merged_df['month'] == month, f'D{month}'] = 1

#Adding Time variable
merged_df['time'] = range(1, len(merged_df) + 1)

merged_df = merged_df.drop(columns=['month','year'])

In [37]:
new_column_names = [
    'Anomaly_adjusted_deg_C',
    'NOAA_CH4',
    'Passenger_Count_adjusted',
    'Total_ET12_adjusted',
    'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'time'
]

# Apply the new column names to the DataFrame
merged_df.columns = new_column_names

In [38]:
# reading the basic variables
MSTA = merged_df['Anomaly_adjusted_deg_C']
ET12 = merged_df["Total_ET12_adjusted"]
CH4 = merged_df['NOAA_CH4']
GMAF = merged_df["Passenger_Count_adjusted"]

D1 = merged_df["D1"]
D2 = merged_df["D2"]
D3 = merged_df["D3"]
D4 = merged_df["D4"]
D5 = merged_df["D5"]
D6 = merged_df["D6"]
D7 = merged_df["D7"]
D8 = merged_df["D8"]
D9 = merged_df["D9"]
D10 = merged_df["D10"]
D11 = merged_df["D11"]
D12 = merged_df["D12"]
time = merged_df["time"]

In [39]:
# Reading the original MSTA time series for all the 348 periods
MSTAfull0 = pd.DataFrame(MSTA_1950_onwards[(MSTA_1950_onwards.index >= '1995-01-01') & (MSTA_1950_onwards.index <= '2023-12-01')])
MSTAfull0.rename(columns={'Anomaly (adjusted deg C)': 'Anomaly_adjusted_deg_C'}, inplace=True)
MSTAfull0

,Anomaly_adjusted_deg_C
Time,
1995-01-01,0.41
1995-02-01,0.70
1995-03-01,0.35
1995-04-01,0.43
1995-05-01,0.26
...,...
2023-08-01,1.18
2023-09-01,1.37
2023-10-01,1.26


### Preparing Fitted and Future values

In [40]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [41]:
#D1-D12 value assigning function:
def monthly_pattern(num_periods, target_month):
    pattern = []
    
    for i in range(num_periods):
        month = (i % 12) + 1  
        if month == target_month:
            pattern.append(1)
        else:
            pattern.append(0)
    return pattern

In [42]:
#Future time order (for time varaiable):
def consecutive_numbers(start_number, num_next):
    return list(range(start_number + 1, start_number + num_next + 1))

In [43]:
#Forecast Explanatory Variables based on Holt-Winters method

# Forecasting for ET12 using Holt-Winters mul method
fit1 = ExponentialSmoothing(ET12, trend='add', seasonal='mul', seasonal_periods=12, damped_trend=True).fit()
fcast1 = fit1.forecast(24).rename("mul seasonal trend for ET12")

# Forecasting for CH4 using Holt-Winters add method
fit2 = ExponentialSmoothing(CH4, trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)
fcast2 = fit2.forecast(24).rename("add seasonal trend for CH4")

# Forecasting for GMAF using Holt-Winters mul method
fit3 = ExponentialSmoothing(GMAF, trend='add', seasonal='mul', seasonal_periods=12, damped_trend=True).fit()
fcast3 = fit3.forecast(24).rename("mul seasonal trend for GMAF")

c:\Users\admin\anaconda3\envs\datasciences1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\envs\datasciences1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\envs\datasciences1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\envs\datasciences1\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [44]:
# putting the fitted values of the forecasts of CH4,GMAF and ET12 in arrays
a1 = np.array(fit1.fittedvalues)
a2 = np.array(fit2.fittedvalues)
a3 = np.array(fit3.fittedvalues)

# putting the values of the forecasts of CH4,GMAF and ET12 in arrays
v1 = np.array(fcast1)
v2 = np.array(fcast2)
v3 = np.array(fcast3)

In [45]:
#Fitted value for D1-12
a4 = monthly_pattern(len(a1), 1)  
a5 = monthly_pattern(len(a1), 2)  
a6 = monthly_pattern(len(a1), 3)  
a7 = monthly_pattern(len(a1), 4)  
a8 = monthly_pattern(len(a1), 5)  # 
a9 = monthly_pattern(len(a1), 6)  
a10 = monthly_pattern(len(a1), 7)  
a11 = monthly_pattern(len(a1), 8)  
a12 = monthly_pattern(len(a1), 9)  
a13 = monthly_pattern(len(a1), 10)  
a14 = monthly_pattern(len(a1), 11)  
a15 = monthly_pattern(len(a1), 12)  

#Forecast value for D1-12
v4 = monthly_pattern(24, 1)  
v5 = monthly_pattern(24, 2)  
v6 = monthly_pattern(24, 3)  
v7 = monthly_pattern(24, 4)  
v8 = monthly_pattern(24, 5)  # 
v9 = monthly_pattern(24, 6)  
v10 = monthly_pattern(24, 7)  
v11 = monthly_pattern(24, 8)  
v12 = monthly_pattern(24, 9)  
v13 = monthly_pattern(24, 10)  
v14 = monthly_pattern(24, 11)  
v15 = monthly_pattern(24, 12)  

In [46]:
#a and v for time
a16 = consecutive_numbers(0, len(a1))
v16 = consecutive_numbers(len(a1), 24)

# Regression Modelling 

In [47]:

from statsmodels.formula.api import ols
# Regression model(s)

formula3 = 'Anomaly_adjusted_deg_C ~ Total_ET12_adjusted + NOAA_CH4 + Passenger_Count_adjusted + D1 + D2 + D3 + D4 + D5 + D6 + D7 + D8 + D9 + D10 + D11 + D12+ time'

# Ordinary Least Squares (OLS)
results3 = ols(formula3, data=merged_df).fit()
print(results3.summary())

                              OLS Regression Results                              
Dep. Variable:     Anomaly_adjusted_deg_C   R-squared:                       0.668
Model:                                OLS   Adj. R-squared:                  0.653
Method:                     Least Squares   F-statistic:                     44.46
Date:                    Fri, 21 Mar 2025   Prob (F-statistic):           4.82e-70
Time:                            08:44:17   Log-Likelihood:                 216.00
No. Observations:                     348   AIC:                            -400.0
Df Residuals:                         332   BIC:                            -338.4
Df Model:                              15                                         
Covariance Type:                nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [48]:
#Extracting the coefficients

b0 = results3.params.Intercept
b1 = results3.params.Total_ET12_adjusted
b2 = results3.params.NOAA_CH4
b3 = results3.params.Passenger_Count_adjusted

# D1 to D12
b4 = results3.params.D1
b5 = results3.params.D2
b6 = results3.params.D3
b7 = results3.params.D4
b8 = results3.params.D5
b9 = results3.params.D6
b10 = results3.params.D7
b11 = results3.params.D8
b12 = results3.params.D9
b13 = results3.params.D10
b14 = results3.params.D11
b15 = results3.params.D12

# Time variable
b16 = results3.params.time

In [ ]:
#Fitted values + Forecast values 

# Initialize the fitted and forecast arrays
F = [0] * 348  # Assuming a1 has 348 periods of data
E = [0] * 24   # Forecast for the next 24 periods

# Calculate the fitted values
for i in range(348):
    fitted_value = b0 + a1[i] * b1 + a2[i] * b2 + a3[i] * b3
    for j in range(4, 17):  # Loop for a4*b4, a5*b5, ..., a16*b16
        fitted_value += globals()[f'a{j}'][i] * globals()[f'b{j}']
    F[i] = fitted_value  # Store the fitted value

# Calculate the forecast values
for i in range(24):
    forecast_value = b0 + v1[i] * b1 + v2[i] * b2 + v3[i] * b3
    for j in range(4, 17):  # Loop for v4*b4, v5*b5, ..., v16*b16
        forecast_value += globals()[f'v{j}'][i] * globals()[f'b{j}']
    E[i] = forecast_value  # Store the forecast value

# Convert the fitted and forecast arrays to DataFrames with the same column name
F = pd.DataFrame(F, columns=['Values'])
E = pd.DataFrame(E, columns=['Values'])

# Generate date ranges for the fitted and forecast values
date_range_fit = pd.date_range(start='1995-01-01', periods=len(F), freq='MS')
date_range_pred = pd.date_range(start='2024-01-01', periods=len(E), freq='MS')

# Set the date ranges as the index for the DataFrames
F.index = date_range_fit
E.index = date_range_pred

# Concatenate F and E along the rows (axis=0)
K = pd.concat([F, E], axis=0)
K

,Values
1995-01-01,0.360866
1995-02-01,0.462107
1995-03-01,0.413664
1995-04-01,0.383024
1995-05-01,0.314422
...,...
2025-08-01,1.067261
2025-09-01,1.054205
2025-10-01,1.065564
2025-11-01,1.102534


In [50]:
# Evaluating the MSE to generate the confidence interval
MSTAfull = pd.DataFrame(MSTAfull0.Anomaly_adjusted_deg_C)
values = MSTAfull[0:348] 
values = pd.DataFrame(values)


MSE_cal = pd.concat([values.reset_index(drop=True), F[0:348].reset_index(drop=True)], axis=1)
MSE_cal["Error"] = MSE_cal["Anomaly_adjusted_deg_C"] - MSE_cal.iloc[:, 1]
MSE = sum(MSE_cal["Error"] ** 2) * 1.0 / len(MSE_cal)
MSE

0.01684510061577715

In [51]:
# Lower and upper bounds of forecasts for 99% confidence interval, z = 2.576
LowerE = E - 2.576*MSE
UpperE = E + 2.576*MSE
LowerE = LowerE.squeeze() 
UpperE = UpperE.squeeze()

In [52]:
#Plotting the forecast result

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Plot the 'Original data' (MSTAfull)
MSTAfull.plot(color='black', label='Original data', ax=ax)

# Plot the 'Forecast Data' (K) 
K.plot(color='red', label='Forecast Data', ax=ax)  # Use label directly here

# Ensure the range is consistent with the forecast length
forecast_dates = K.index[-24:]  # Get the last 24 months for the forecast
ax.fill_between(forecast_dates, LowerE[-24:], UpperE[-24:], color='b', alpha=0.3, label="Confidence Interval")

plt.xlabel('Time')
plt.ylabel('Values')
plt.title('Regression forecast of MSTA')
ax.legend()
plt.tight_layout()
plt.show()

In [53]:
#Assumption Testing/Residual Diagnostic

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import scipy.stats as stats

# Calculate residuals
residuals = results3.resid
fitted_values = results3.fittedvalues

# Create a figure with multiple subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

# Plot residuals over time
axes[0, 0].plot(residuals, label="Residuals")
axes[0, 0].axhline(0, color='black', linestyle='--')
axes[0, 0].set_title("Residuals of OLS3 Model")
axes[0, 0].legend()

# Plot ACF of residuals
plot_acf(residuals, lags=40, ax=axes[0, 1])
axes[0, 1].set_title('ACF of Residuals')

# Plot PACF of residuals
plot_pacf(residuals, lags=40, ax=axes[1, 0])
axes[1, 0].set_title('PACF of Residuals')

# Plot residuals vs. fitted values to check for homoscedasticity
axes[1, 1].scatter(fitted_values, residuals)
axes[1, 1].axhline(0, color='black', linestyle='--')
axes[1, 1].set_xlabel('Fitted Values')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Residuals vs. Fitted Values')

# Plot histogram of residuals
sns.histplot(residuals, kde=True, ax=axes[2, 0])
axes[2, 0].set_title('Histogram of Residuals')
axes[2, 0].set_xlabel('Residuals')
axes[2, 0].set_ylabel('Frequency')

# Q-Q plot of residuals
stats.probplot(residuals, dist="norm", plot=axes[2, 1])
axes[2, 1].set_title('Q-Q Plot of Residuals')

# Adjust layout for better visualization
plt.tight_layout()

# Display the plot
plt.show()